In [ ]:
import sys, os, pathlib
sys.path.append("/root/shared/gitrepos/smart-comp-sci/utils")
import smart_analysis
from matplotlib import pyplot as plt
import numpy as np
plt.style.use(str(pathlib.Path.cwd() / ".." / "utils" / "smart_plots.mplstyle"))

Analyze CRU data with vs. without SERCA (change paths to data as necessary).

In [ ]:
npy_dir = pathlib.Path.cwd() / ".." / "analysis_data" / "cru" / "cru_npy_files"
cru_results_folder = str(pathlib.Path.cwd() / ".." / "analysis_data" / "cru" / "cru-results")
# cru_results_folder_noSERCA = str(pathlib.Path.cwd() / ".." / "analysis_data" / "calcium_results" / "cru-results-noserca")
cru_results_folder_noSERCA = str(pathlib.Path.cwd() / ".." / "analysis_data" / "cru" / "cru-results-noserca")
cru_mesh = str(pathlib.Path.cwd() / ".." / "analysis_data" / "cru" / "cru_mesh.h5")
results_folders = [cru_results_folder, cru_results_folder, cru_results_folder,
                   cru_results_folder_noSERCA, cru_results_folder_noSERCA, cru_results_folder_noSERCA]
tests = ["cru-all", "cru-center", "cru-edge", 
         "cru-all-noSERCA", "cru-center-noSERCA", "cru-edge-noSERCA"]
cru_center = [-200, -100, -150, 200, 150, 300]
cru_edge = [-1000, -1000, -1000, -600, 1000, 1000]
domains = [[], cru_center, cru_edge, [], cru_center, cru_edge]
for i in range(len(tests)):
    results_folder = results_folders[i]
    tVec, results_all = smart_analysis.analyze_all(
        mesh_file=cru_mesh, results_path=results_folder, display=False,
        subdomain=domains[i])
    results_all.insert(0, tVec) # add time as first element in list
    max_length = len(tVec)
    for j in range(len(results_all)):
        if len(results_all[j]) > max_length:
            max_length = len(results_all[j])
    for j in range(len(results_all)):
        num_zeros = max_length - len(results_all[j])
        for k in range(num_zeros):
            results_all[j].append(0)
    np.save(npy_dir / f"{tests[i]}.npy", results_all)

Make plots shown in Figure 4.

In [ ]:
npy_dir = pathlib.Path.cwd() / ".." / "analysis_data" / "cru" / "cru_npy_files"
cru_vars = ["ATP", "Ca", "CaSR", "CMDN", "CSQN", "RyR", "SERCA", "TRPN"]
plot_cru = "Ca"
f, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=False, figsize=(4,6))
plot_cru1 = "Ca"
results_cru_withSERCA = np.load(f"{npy_dir}/cru-center.npy")
results_cru_noSERCA = np.load(f"{npy_dir}/cru-center-noSERCA.npy")
cru_idx1 = cru_vars.index(plot_cru1) + 1
ax1.plot(results_cru_withSERCA[0], results_cru_withSERCA[cru_idx1],label="CRU - with SERCA")
ax1.plot(results_cru_noSERCA[0], results_cru_noSERCA[cru_idx1],label="CRU - no SERCA")
ax1.set_ylim([0, 6])
ax1.set_xlim([-.01, 0.1])
# ax1.legend()
ax1.set_ylabel("Calcium (μM)")
ax1.set_xlabel("Time (s)")

results_cru_withSERCA_edge = np.load(f"{npy_dir}/cru-edge.npy")
results_cru_noSERCA_edge = np.load(f"{npy_dir}/cru-edge-noSERCA.npy")
cru_idx1 = cru_vars.index(plot_cru1) + 1
ax2.plot(results_cru_withSERCA_edge[0], 
         results_cru_withSERCA_edge[cru_idx1],label="CRU - with SERCA")
ax2.plot(results_cru_noSERCA_edge[0], 
         results_cru_noSERCA_edge[cru_idx1],label="CRU - no SERCA")
ax2.set_ylim([0, 6])
ax2.set_xlim([-.01, 0.1])
ax2.set_ylabel("Calcium (μM)")
ax2.set_xlabel("Time (s)")

plot_cru2 = "CaSR"
cru_idx2 = cru_vars.index(plot_cru2) + 1
ax3.plot(results_cru_withSERCA[0], results_cru_withSERCA[cru_idx2],label="CRU - with SERCA")
ax3.plot(results_cru_noSERCA[0], results_cru_noSERCA[cru_idx2],label="CRU - no SERCA")
# ax3.legend()
ax3.set_ylabel("SR calcium (μM)")
ax3.set_ylim([0, 1400])
ax3.set_xlim([-.01, 0.1])
ax3.set_xlabel("Time (s)")
plt.savefig(f"{npy_dir}/cru_plot.pdf", format="pdf")